In [95]:
from scipy.stats import entropy
import pandas as pd
import numpy as np
import malaya_speech
import IPython.display as ipd

In [96]:
p1 = 'submit_valid_acc_0.9125_後面接svc排行榜0.963955516.csv'
p2 = 'submit_valid_acc_0.9333333333333333_後面接logstic_排行榜0.9662309294.csv'
p3 = 'submit_valid_acc_0.9111111111111111_這個在分群上分的蠻好的_排行榜0.9683586426.csv'
p4 = 'senet20_20210605_random_state529_validacc_0.9333333333333333_排行榜0.9646076361.csv'

def caculate_entropy(p):
    df = pd.read_csv(p)
    e = entropy(df.iloc[:10000,1:].values,axis=1)
    return np.mean(e)

In [97]:
ent = []
for i,p in enumerate([p1,p2,p3,p4]):
    print(i,caculate_entropy(p))
    ent.append(caculate_entropy(p))
ent

0 0.5219590919849224
1 0.45633177481680215
2 0.24342260577891348
3 0.5048804796212653


[0.5219590919849224,
 0.45633177481680215,
 0.24342260577891348,
 0.5048804796212653]

In [98]:
from scipy.special import softmax
ent = softmax(ent)
ent = 1/ent #ent愈大預測品質愈糟所以要取倒數換成weight,weight愈大預測品質愈好
weight = softmax(ent)
weight

array([0.15138351, 0.19425744, 0.49307872, 0.16128034])

In [99]:
weight.sum()

1.0000000000000002

In [100]:
df1 = pd.read_csv(p1)
df2 = pd.read_csv(p2)
df3 = pd.read_csv(p3)
df4 = pd.read_csv(p4)
new_df = df1.copy()
new_df.iloc[:,1:] = df1.iloc[:,1:]+df2.iloc[:,1:]+df3.iloc[:,1:]+df4.iloc[:,1:]
new_df.iloc[:,1:] = df1.iloc[:,1:]*weight[0] + df2.iloc[:,1:]*weight[1] + df3.iloc[:,1:]*weight[2] + df4.iloc[:,1:]*weight[3]

In [101]:
new_df

,Filename,Barking,Howling,Crying,COSmoke,GlassBreaking,Other
0,public_00001,0.005391,0.000754,0.000896,0.007578,0.739067,0.246314
1,public_00002,0.072565,0.000532,0.043385,0.181576,0.023347,0.678594
2,public_00003,0.959305,0.002581,0.003290,0.001100,0.000953,0.032771
3,public_00004,0.026162,0.000622,0.012113,0.019630,0.147697,0.793776
4,public_00005,0.334816,0.016087,0.577754,0.005930,0.007812,0.057602
...,...,...,...,...,...,...,...
29995,private_19996,0.166609,0.166609,0.166609,0.166609,0.166609,0.166955
29996,private_19997,0.166609,0.166609,0.166609,0.166609,0.166609,0.166955
29997,private_19998,0.166609,0.166609,0.166609,0.166609,0.166609,0.166955
29998,private_19999,0.166609,0.166609,0.166609,0.166609,0.166609,0.166955


In [102]:
new_df.sum(axis=1)

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
29995    1.0
29996    1.0
29997    1.0
29998    1.0
29999    1.0
Length: 30000, dtype: float64

In [103]:
len(new_df)

30000

In [104]:
new_df.to_csv('Voteing_V8_entropy_vote.csv',index=False)